# DEAM Dataset - Feed Forward Neural Network
## Essentia Best Overall & openSMILE GeMAPS Featureset

## Import relevant libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torcheval.metrics import R2Score

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import math

import sys
sys.path.insert(1, '../../utils')
from paths import *

## Neural Network Training

### Import annotations dataset

In [2]:
df_annotations = pd.read_csv(get_deam_path('processed/annotations/deam_static_annotations.csv'))
df_annotations

,song_id,valence_mean_mapped,arousal_mean_mapped
0,2,-0.475,-0.500
1,3,-0.375,-0.425
2,4,0.175,0.125
3,5,-0.150,0.075
4,7,0.200,0.350
...,...,...,...
1739,1996,-0.275,0.225
1740,1997,0.075,-0.275
1741,1998,0.350,0.300
1742,1999,-0.100,0.100


### Import the featureset

In [3]:
df_essentia_best_overall_opensmile_gemaps_features = pd.read_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_gemaps_features.csv'))

# drop Unnamed:0 column
df_essentia_best_overall_opensmile_gemaps_features = df_essentia_best_overall_opensmile_gemaps_features[df_essentia_best_overall_opensmile_gemaps_features.columns[1:]]

df_essentia_best_overall_opensmile_gemaps_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [4]:
df_essentia_best_overall_opensmile_gemaps_features.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 199 columns):
 #    Column                                          Dtype  
---   ------                                          -----  
 0    song_id                                         int64  
 1    lowlevel.melbands_kurtosis.dmean                float64
 2    lowlevel.melbands_kurtosis.dmean2               float64
 3    lowlevel.melbands_kurtosis.dvar                 float64
 4    lowlevel.melbands_kurtosis.dvar2                float64
 5    lowlevel.melbands_kurtosis.max                  float64
 6    lowlevel.melbands_kurtosis.mean                 float64
 7    lowlevel.melbands_kurtosis.median               float64
 8    lowlevel.melbands_kurtosis.min                  float64
 9    lowlevel.melbands_kurtosis.stdev                float64
 10   lowlevel.melbands_kurtosis.var                  float64
 11   lowlevel.melbands_skewness.dmean                float64
 12   lowlevel.melbands_

Join both the featureset and annotation set together

In [5]:
df_essentia_best_overall_opensmile_gemaps_whole = pd.merge(df_essentia_best_overall_opensmile_gemaps_features, df_annotations, how='inner', on='song_id')
df_essentia_best_overall_opensmile_gemaps_whole = df_essentia_best_overall_opensmile_gemaps_whole.drop('song_id', axis=1)
df_essentia_best_overall_opensmile_gemaps_whole

,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,lowlevel.melbands_kurtosis.var,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,valence_mean_mapped,arousal_mean_mapped
0,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,1390.829224,...,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573,-0.475,-0.500
1,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,3008.853760,...,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012,-0.375,-0.425
2,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,387.776367,...,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689,0.175,0.125
3,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,253.805130,...,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987,-0.150,0.075
4,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,7165.564941,...,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756,0.200,0.350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,127.879837,...,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377,-0.275,0.225
1740,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,145.073792,...,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021,0.075,-0.275
1741,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,394.295044,...,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816,0.350,0.300
1742,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,296.105286,...,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784,-0.100,0.100


#### Prepare dataframes for the neural network

Perform splitting of the dataframe into training and testing sets

In [6]:
features = df_essentia_best_overall_opensmile_gemaps_features.drop('song_id', axis=1)
features

,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,lowlevel.melbands_kurtosis.var,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,1390.829224,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,3008.853760,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,387.776367,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,253.805130,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,7165.564941,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,127.879837,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,145.073792,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,394.295044,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,296.105286,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [7]:
targets = df_annotations.drop('song_id', axis=1)
targets

,valence_mean_mapped,arousal_mean_mapped
0,-0.475,-0.500
1,-0.375,-0.425
2,0.175,0.125
3,-0.150,0.075
4,0.200,0.350
...,...,...
1739,-0.275,0.225
1740,0.075,-0.275
1741,0.350,0.300
1742,-0.100,0.100


Perform 80-20 train-test split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

Create tensors for X_train and X_test

In [9]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float64)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float64)

Create tensors for Y_train and Y_test

In [10]:
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float64)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float64)

Define neural network parameters and instantitate neural network

In [11]:
input_size = 1
hidden_size = 20 
output_size = 2  # Output size for valence and arousal
learning_rate = 0.001
criterion = nn.MSELoss()
num_epochs = 300

Define the neural network

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), 2)
        )

    def forward(self, x):
        return self.layers(x)

#### Training

Prepare input_train_data and target_train_labels

In [13]:
input_train_data = X_train_tensor.float()

# input_train_data = input_train_data.view(input_train_data.shape[1], -1)
print(input_train_data.shape)

target_train_labels = y_train_tensor

torch.Size([1395, 198])


Training loop

In [14]:
model = NeuralNetwork(input_size=input_train_data.shape[1])
optimiser = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  optimiser.zero_grad()
  
  # forward pass
  output = model(input_train_data)

  # calculate loss
  loss = torch.sqrt(criterion(output.float(), target_train_labels.float()))

  # backward pass
  loss.backward()
  # update weights
  optimiser.step()

  print(f'Epoch {epoch + 1}, Loss: {math.sqrt(loss.item())}')

print("Training completed.")

Epoch 1, Loss: 5.359856675637327
Epoch 2, Loss: 4.08793541691896
Epoch 3, Loss: 2.9073185750786017
Epoch 4, Loss: 2.4615248206338354
Epoch 5, Loss: 2.1695634355310878
Epoch 6, Loss: 1.9508276894525913
Epoch 7, Loss: 1.549972130924836
Epoch 8, Loss: 1.0984292346106168
Epoch 9, Loss: 0.9660001612380784
Epoch 10, Loss: 0.743966990961487
Epoch 11, Loss: 0.9568273521734305
Epoch 12, Loss: 1.021335043850538
Epoch 13, Loss: 1.0379335393852471
Epoch 14, Loss: 0.9317494774651399
Epoch 15, Loss: 0.8192123301660303
Epoch 16, Loss: 0.6342466139892846
Epoch 17, Loss: 0.6182557771692789
Epoch 18, Loss: 0.6128060024136825
Epoch 19, Loss: 0.569193879557226
Epoch 20, Loss: 0.6109470727663247
Epoch 21, Loss: 0.5800469643800055
Epoch 22, Loss: 0.5676425989107178
Epoch 23, Loss: 0.5642295315350718
Epoch 24, Loss: 0.5670332233797215
Epoch 25, Loss: 0.5866851106186405
Epoch 26, Loss: 0.5640428629492379
Epoch 27, Loss: 0.5757337294369508
Epoch 28, Loss: 0.5872606215898828
Epoch 29, Loss: 0.5687865654226637
E

## Neural Network Testing

Prepare input_test_data and target_test_labels

In [15]:
input_test_data = X_test_tensor.float()

# input_test_data = input_test_data.view(input_test_data.shape[1], -1)
print(input_test_data.shape)

target_test_labels = y_test_tensor

torch.Size([349, 198])


Generating scores

In [16]:
with torch.no_grad():
  test_pred = model(input_test_data)
  test_loss = criterion(test_pred.float(), target_test_labels)

print(f'Test RMSE: {math.sqrt(test_loss.item())}')

metric = R2Score()
metric.update(test_pred, target_test_labels)
r2_score = metric.compute()
print(f'Test R^2 score: {r2_score.item()}')


Test RMSE: 0.23002626564123974
Test R2 score: 0.4238930194007332


True values (test set)

In [17]:
target_test_labels

tensor([[-0.1500, -0.1500],
        [-0.3000, -0.1000],
        [ 0.2000,  0.3500],
        [ 0.2250,  0.4500],
        [-0.1750, -0.2000],
        [-0.5250, -0.3000],
        [-0.2500, -0.7750],
        [ 0.3000,  0.3000],
        [-0.1750, -0.4000],
        [ 0.4500,  0.1500],
        [ 0.1750,  0.0250],
        [-0.1750, -0.0250],
        [-0.0500, -0.3000],
        [ 0.1250,  0.3000],
        [-0.0750, -0.1500],
        [-0.2000, -0.2750],
        [-0.6000, -0.2250],
        [ 0.1500, -0.2000],
        [ 0.2750,  0.6000],
        [-0.1500, -0.4500],
        [-0.2250, -0.6250],
        [-0.0250, -0.4500],
        [-0.5250, -0.1250],
        [ 0.0000,  0.3250],
        [ 0.1250,  0.3750],
        [ 0.1500, -0.2500],
        [ 0.4500,  0.3250],
        [ 0.2500,  0.2250],
        [-0.1000,  0.0750],
        [ 0.4250,  0.1250],
        [-0.4500, -0.3500],
        [-0.0500,  0.3750],
        [-0.4750, -0.2000],
        [-0.2750, -0.4000],
        [-0.4000, -0.2250],
        [ 0.1000, -0

Predicted values

In [18]:
test_pred

tensor([[ 0.0547,  0.0551],
        [-0.2431, -0.3002],
        [ 0.3554,  0.2829],
        [ 0.0381,  0.0344],
        [-0.0161, -0.0211],
        [-0.3246, -0.3934],
        [-0.3862, -0.4560],
        [ 0.3420,  0.2847],
        [-0.2490, -0.3087],
        [ 0.2706,  0.2573],
        [ 0.1359,  0.1556],
        [-0.1724, -0.2084],
        [-0.3245, -0.3781],
        [ 0.2812,  0.2641],
        [-0.1526, -0.1605],
        [-0.1334, -0.1549],
        [-0.3463, -0.4098],
        [-0.2318, -0.2878],
        [ 0.2612,  0.2440],
        [-0.2573, -0.3358],
        [-0.2339, -0.2825],
        [-0.2606, -0.3271],
        [-0.1466, -0.1760],
        [ 0.1977,  0.2046],
        [ 0.2785,  0.2620],
        [-0.1713, -0.2116],
        [ 0.0926,  0.0971],
        [ 0.0874,  0.0884],
        [ 0.2230,  0.2293],
        [ 0.2898,  0.2685],
        [-0.2282, -0.2787],
        [ 0.0668,  0.0664],
        [ 0.1905,  0.1981],
        [-0.1213, -0.1451],
        [-0.3215, -0.3580],
        [-0.0042, -0